In [414]:
#importing modules
import pandas as pd
from pathlib import Path

In [415]:
#setting file path to read
voc_path = Path("Resources/voc_rehab.csv")
unemployment_path = Path("Resources/unemployment.csv")
success_path = Path('Resources/successful_closures.csv')

# Read with Pandas
vocation_df = pd.read_csv(voc_path)
vocation_df.head()

,Year,County,Sex,Number of Consumers,Annotation Code,Count Annotation Desc,Percent,Percent Annotation Desc
0,2014,Alameda,Not Reported,4,NaN,NaN,NaN,NaN
1,2014,Alameda,Female,"2,356",NaN,NaN,NaN,NaN
2,2014,Alameda,Male,"2,391",NaN,NaN,NaN,NaN
3,2014,Alpine,Female,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...
4,2014,Amador,Female,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...


In [416]:
#shows what columns to determine relevance for vocational data
vocation_df.columns

Index(['Year', 'County', 'Sex', 'Number of Consumers', 'Annotation Code',
       'Count Annotation Desc', 'Percent', 'Percent Annotation Desc'],
      dtype='object')

In [417]:
#removing irrelevant columns
del vocation_df["Annotation Code"]
del vocation_df["Count Annotation Desc"]
del vocation_df["Percent"]
del vocation_df["Percent Annotation Desc"]
del vocation_df["Sex"]

#removing commas from numerical data
vocation_df = vocation_df.replace(',', '', regex=True)
vocation_df.head()

,Year,County,Number of Consumers
0,2014,Alameda,4
1,2014,Alameda,2356
2,2014,Alameda,2391
3,2014,Alpine,NaN
4,2014,Amador,NaN


In [418]:
#filling na values w/0
vocation_df = vocation_df.fillna(0)

#filtering data by year
vocation_df = vocation_df.set_index('Year')

#changing 'number of consumers' to type int
vocation_df = vocation_df.astype({'Number of Consumers': int})

vocation_df.head()

,County,Number of Consumers
Year,,
2014,Alameda,4
2014,Alameda,2356
2014,Alameda,2391
2014,Alpine,0
2014,Amador,0


In [419]:
#adding together total number of consumers per coutny per year
vocation_df = vocation_df.groupby(['Year', 'County'])['Number of Consumers'].sum().reset_index()
vocation_df

,Year,County,Number of Consumers
0,2014,Alameda,4751
1,2014,Alpine,0
2,2014,Amador,18
3,2014,Butte,735
4,2014,Calaveras,30
...,...,...,...
235,2017,Tulare,1117
236,2017,Tuolumne,87
237,2017,Ventura,2878
238,2017,Yolo,499


In [466]:
#introducing unemployment data
unemployment_df = pd.read_csv(unemployment_path)
unemployment_df = unemployment_df.set_index('Area Type')
unemployment_df

# Filter Area Type to only include County and State data and reduce columns
unemployment_df = pd.read_csv(unemployment_path)

#changing column names to match vocational data
unemployment_df = unemployment_df.rename(columns={'Area Name': 'County'})

#removing 'County' from county names
unemployment_df = unemployment_df.replace(' County', '', regex=True)

unemployment_df = unemployment_df.set_index('Area Type')
unemployment_df

# Filter Area Type to only include County and State data and reduce columns
area_type_df = unemployment_df.loc[['State','County'],
                                   ['County', 'Year', 'Labor Force', 'Employment', 'Unemployment', 'Unemployment Rate']]
area_type_df

# Group by Year and calculate averages
year_grouped = area_type_df.groupby(['Year', 'County']).mean()
year_grouped

# Reduce to only 2014-2017 to line up with vocational data
recent = year_grouped.loc[[2014, 2015, 2016, 2017]]
recent

Labor Force     Employment  Unemployment  Unemployment Rate
Year County                                                                  
2014 Alameda    806325.000000  758808.333333  47500.000000           0.059000
     Alpine        509.166667     466.666667     41.666667           0.083167
     Amador      14322.500000   13118.333333   1204.166667           0.083917
     Butte      101058.333333   92208.333333   8841.666667           0.087500
     Calaveras   20204.166667   18569.166667   1635.000000           0.081083
...                       ...            ...           ...                ...
2017 Tulare     204458.333333  182966.666667  21491.666667           0.105167
     Tuolumne    21369.166667   20207.500000   1160.833333           0.054250
     Ventura    423675.000000  404475.000000  19225.000000           0.045333
     Yolo       106558.333333  101091.666667   5483.333333           0.051500
     Yuba        28716.666667   26566.666667   2141.666667           0.074667

[236 rows x 4 columns]

In [467]:
#introducing successful vocational services
success_df = pd.read_csv(success_path)
success_df

,Year,County,Occupation,Successful Closures,Annotation Code,Count Annotation Desc,Percent,Percent Annotation Desc
0,2014,Alameda,Not Reported,67.0,NaN,NaN,NaN,NaN
1,2014,Alameda,Clerical and Administrative Support,114.0,NaN,NaN,NaN,NaN
2,2014,Alameda,Community and Social Service Occupations,3.0,NaN,NaN,NaN,NaN
3,2014,Alameda,Computer and Mathematical Occupations,5.0,NaN,NaN,NaN,NaN
4,2014,Alameda,"Education, Training, and Library Occupations",4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1935,2017,Yuba,"Professional, Paraprofessional and Technical",NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...
1936,2017,Yuba,Protective Service Occupations,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...
1937,2017,Yuba,RSA Special Occupations and Miscellaneous,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...
1938,2017,Yuba,Sales and Related Occupations,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...


In [468]:
#displaying columns to display relevance
success_df.columns

Index(['Year', 'County', 'Occupation', 'Successful Closures',
       'Annotation Code', 'Count Annotation Desc', 'Percent',
       'Percent Annotation Desc'],
      dtype='object')

In [469]:
#removing irrelevant columns
del success_df['Occupation']
del success_df['Annotation Code']
del success_df['Count Annotation Desc']
del success_df['Percent']
del success_df['Percent Annotation Desc']

#replacing na w/ value of 0
success_df = success_df.fillna(0)
success_df

,Year,County,Successful Closures
0,2014,Alameda,67.0
1,2014,Alameda,114.0
2,2014,Alameda,3.0
3,2014,Alameda,5.0
4,2014,Alameda,4.0
...,...,...,...
1935,2017,Yuba,0.0
1936,2017,Yuba,0.0
1937,2017,Yuba,0.0
1938,2017,Yuba,0.0


In [470]:
#adding total number of successful closures per county per year
success_df = success_df.groupby(['Year', 'County'])['Successful Closures'].sum().reset_index()
success_df

,Year,County,Successful Closures
0,2014,Alameda,554.0
1,2014,Amador,0.0
2,2014,Butte,113.0
3,2014,Calaveras,1.0
4,2014,Colusa,0.0
...,...,...,...
225,2017,Tulare,75.0
226,2017,Tuolumne,0.0
227,2017,Ventura,301.0
228,2017,Yolo,50.0


In [471]:
#merging together vocational data
merged_vocation_df = pd.merge(vocation_df, success_df, on=['Year', 'County'], how='inner')
merged_vocation_df

,Year,County,Number of Consumers,Successful Closures
0,2014,Alameda,4751,554.0
1,2014,Amador,18,0.0
2,2014,Butte,735,113.0
3,2014,Calaveras,30,1.0
4,2014,Colusa,0,0.0
...,...,...,...,...
223,2017,Tulare,1117,75.0
224,2017,Tuolumne,87,0.0
225,2017,Ventura,2878,301.0
226,2017,Yolo,499,50.0


In [472]:
#organized data by year by county
merged_vocation_df = merged_vocation_df.set_index(['Year', 'County'])
merged_vocation_df

Number of Consumers  Successful Closures
Year County                                             
2014 Alameda                   4751                554.0
     Amador                      18                  0.0
     Butte                      735                113.0
     Calaveras                   30                  1.0
     Colusa                       0                  0.0
...                             ...                  ...
2017 Tulare                    1117                 75.0
     Tuolumne                    87                  0.0
     Ventura                   2878                301.0
     Yolo                       499                 50.0
     Yuba                       211                  1.0

[228 rows x 2 columns]

In [476]:
#merging all data frames into one and adjusting formating
master_df = pd.merge(merged_vocation_df, recent, on=['Year', 'County'], how='left')
pd.options.display.float_format = '{:,.2f}'.format
master_df

Number of Consumers  Successful Closures  Labor Force  \
Year County                                                             
2014 Alameda                   4751               554.00   806,325.00   
     Amador                      18                 0.00    14,322.50   
     Butte                      735               113.00   101,058.33   
     Calaveras                   30                 1.00    20,204.17   
     Colusa                       0                 0.00    11,079.17   
...                             ...                  ...          ...   
2017 Tulare                    1117                75.00   204,458.33   
     Tuolumne                    87                 0.00    21,369.17   
     Ventura                   2878               301.00   423,675.00   
     Yolo                       499                50.00   106,558.33   
     Yuba                       211                 1.00    28,716.67   

                Employment  Unemployment  Unemployment Rate  
Year County                                                  
2014 Alameda    758,808.33     47,500.00               0.06  
     Amador      13,118.33      1,204.17               0.08  
     Butte       92,208.33      8,841.67               0.09  
     Calaveras   18,569.17      1,635.00               0.08  
     Colusa       9,132.50      1,949.17               0.18  
...                    ...           ...                ...  
2017 Tulare     182,966.67     21,491.67               0.11  
     Tuolumne    20,207.50      1,160.83               0.05  
     Ventura    404,475.00     19,225.00               0.05  
     Yolo       101,091.67      5,483.33               0.05  
     Yuba        26,566.67      2,141.67               0.07  

[228 rows x 6 columns]